In [12]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device=device)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define transforms (CIFAR-100 images are 3-channel RGB, so we'll convert to grayscale for the LeNet input)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert RGB to grayscale
    transforms.Resize((32, 32)),  # Resize to 32x32 if needed
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize grayscale images (mean, std)
])

# Load CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Now your model can be trained with train_loader and evaluated with test_loader


Files already downloaded and verified
Files already downloaded and verified


In [8]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20  # Set the number of epochs for training

for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    model.train()  # Set the model to training mode

    for i, data in enumerate(train_loader, 0):
        # Get the inputs and move them to the GPU if available
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Finished Training')


Epoch [1/20], Step [100/1563], Loss: 2.2294
Epoch [1/20], Step [200/1563], Loss: 2.0515
Epoch [1/20], Step [300/1563], Loss: 1.8967
Epoch [1/20], Step [400/1563], Loss: 1.7837
Epoch [1/20], Step [500/1563], Loss: 1.7239
Epoch [1/20], Step [600/1563], Loss: 1.7259
Epoch [1/20], Step [700/1563], Loss: 1.6775
Epoch [1/20], Step [800/1563], Loss: 1.6669
Epoch [1/20], Step [900/1563], Loss: 1.6354
Epoch [1/20], Step [1000/1563], Loss: 1.6249
Epoch [1/20], Step [1100/1563], Loss: 1.6530
Epoch [1/20], Step [1200/1563], Loss: 1.6037
Epoch [1/20], Step [1300/1563], Loss: 1.6188
Epoch [1/20], Step [1400/1563], Loss: 1.5667
Epoch [1/20], Step [1500/1563], Loss: 1.5415
Epoch [2/20], Step [100/1563], Loss: 1.5285
Epoch [2/20], Step [200/1563], Loss: 1.5206
Epoch [2/20], Step [300/1563], Loss: 1.4896
Epoch [2/20], Step [400/1563], Loss: 1.4862
Epoch [2/20], Step [500/1563], Loss: 1.4922
Epoch [2/20], Step [600/1563], Loss: 1.4875
Epoch [2/20], Step [700/1563], Loss: 1.4548
Epoch [2/20], Step [800/15

In [9]:
correct = 0
total = 0
model.eval()  # Set the model to evaluation mode (no gradient computation)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Accuracy on test set: 61.73%


In [10]:
def apply_pruning(model, method='global', amount=0.2):
    parameters_to_prune = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, 'weight'))
    
    if method == 'global':
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method=prune.L1Unstructured,
            amount=amount,
        )
    elif method == 'l1_unstructured':
        for module, param in parameters_to_prune:
            prune.l1_unstructured(module, name=param, amount=amount)
    elif method == 'random_unstructured':
        for module, param in parameters_to_prune:
            prune.random_unstructured(module, name=param, amount=amount)

In [11]:
torch.save(model.state_dict(),"LeNet.pt")

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import torch
import torch.nn.utils.prune as prune

# Function to calculate the sparsity of the pruned layers
def calculate_sparsity(model):
    total_params = 0
    pruned_params = 0
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            total_params += module.weight.nelement()  # Total number of weights
            pruned_params += torch.sum(module.weight == 0).item()  # Pruned (zeroed-out) weights

    return 100 * pruned_params / total_params


# Test and print results for each pruning method
pruning_methods = ['global', 'l1_unstructured', 'random_unstructured']

for method in pruning_methods:
    # Reset the model
    
    # Apply pruning
    print(f"\nApplying {method} pruning:")
    apply_pruning(model, method=method, amount=0.2)

    # Calculate and print sparsity
    sparsity = calculate_sparsity(model)
    print(f"Sparsity after {method} pruning: {sparsity:.2f}%")

    # Optional: Inspect pruned weights in one of the layers (e.g., first conv layer)
    print(f"Sample pruned weights in conv1 layer after {method} pruning:")
    print(model.conv1.weight)
